In [1]:
# Uncomment to install/upgrade ras-commander from pip
#!pip install --upgrade ras-commander

#Import the ras-commander package
from ras_commander import *

##### Optional Code Cell For Development/Testing Mode (Local Copy)
##### Uncomment and run this cell instead of the pip cell above

```python
# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander from local dev copy")
from ras_commander import *
```

In [2]:
import os
import sys
from pathlib import Path

In [3]:
# If /example_projects/BaldEagleCrkMulti2D does not exist, extract it
project_path = "./example_projects/BaldEagleCrkMulti2D"

if not os.path.exists(project_path):
    project_path = RasExamples.extract_project("BaldEagleCrkMulti2D")

# Initialize the RAS project
init_ras_project(project_path, "6.6")

# Define the plan to work with
plan_number = "06"



2025-11-17 15:41:15 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.rasmap


In [4]:
import subprocess
import sys

# Print instructions to the user
print("\n" + "="*60)
print("MANUAL STEP REQUIRED: Update .rasmap to Version 6.x")
print("This project was created in HEC-RAS 5.0.7. To generate stored maps in HEC-RAS 6.x, you must:")
print("1. HEC-RAS will now be opened with this project.")
print("2. In HEC-RAS, open RAS Mapper (from the main toolbar).") 
print("3. When prompted, allow RAS Mapper to update the .rasmap file to the new version.")
print("4. Once the update is complete, close RAS Mapper and exit HEC-RAS.")
print("\nAfter closing HEC-RAS, return here and continue running the notebook.")
print("="*60 + "\n")

ras_exe = ras.ras_exe_path
prj_path = f'"{str(ras.prj_file)}"'  # Add quotes around project path

command = f"{ras_exe} {prj_path}"
print(command)

try:
    # Capture the process object so we can get its PID
    if sys.platform == "win32":
        hecras_process = subprocess.Popen(command)
    else:
        hecras_process = subprocess.Popen([ras_exe, prj_path])
    
    # Store the process ID for use in the next cell
    hecras_pid = hecras_process.pid
    print(f"Opened HEC-RAS with Process ID: {hecras_pid}")
    print("Please wait for the next cell to automate RAS Mapper...")
    
except Exception as e:
    print(f"Failed to launch HEC-RAS: {e}")
    hecras_pid = None


MANUAL STEP REQUIRED: Update .rasmap to Version 6.x
This project was created in HEC-RAS 5.0.7. To generate stored maps in HEC-RAS 6.x, you must:
1. HEC-RAS will now be opened with this project.
2. In HEC-RAS, open RAS Mapper (from the main toolbar).
3. When prompted, allow RAS Mapper to update the .rasmap file to the new version.
4. Once the update is complete, close RAS Mapper and exit HEC-RAS.

After closing HEC-RAS, return here and continue running the notebook.

C:\Program Files (x86)\HEC\HEC-RAS\6.6\Ras.exe "C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.prj"
Opened HEC-RAS with Process ID: 398092
Please wait for the next cell to automate RAS Mapper...


In [5]:
!pip install pywinauto

In [6]:
# This cell will inspect the HEC-RAS window and list all interactable elements.
# It also provides an example of how to use the pywinauto library for more robust automation,
# including interaction with 64-bit processes like RAS Mapper.

import win32gui
import win32con
import win32api
import win32process
import time
import ctypes
from ctypes import wintypes

# ==============================================================================
# Part 1: Inspecting the HEC-RAS Window with pywin32
# ==============================================================================
# This section uses the original pywin32 approach to list all menus and
# child controls (buttons, text boxes, etc.) of the main HEC-RAS window.

# Constants from original script
MF_BYPOSITION = 0x00000400

def get_windows_by_pid(pid):
    """Find all windows belonging to a specific process ID"""
    def callback(hwnd, hwnds):
        if win32gui.IsWindowVisible(hwnd) and win32gui.IsWindowEnabled(hwnd):
            _, window_pid = win32process.GetWindowThreadProcessId(hwnd)
            if window_pid == pid:
                window_title = win32gui.GetWindowText(hwnd)
                if window_title:
                    hwnds.append((hwnd, window_title))
        return True
    hwnds = []
    win32gui.EnumWindows(callback, hwnds)
    return hwnds

def find_main_hecras_window(windows):
    """Find the main HEC-RAS window from a list of windows"""
    for hwnd, title in windows:
        if "HEC-RAS" in title and win32gui.GetMenu(hwnd):
            return hwnd, title
    return None, None

def get_menu_string(menu_handle, pos):
    """Get menu item string at position"""
    buf_size = 256
    buf = ctypes.create_unicode_buffer(buf_size)
    user32 = ctypes.windll.user32
    result = user32.GetMenuStringW(menu_handle, pos, buf, buf_size, MF_BYPOSITION)
    if result:
        return buf.value
    return ""

def enumerate_all_menus(hwnd):
    """Enumerate all menus and their items in great detail."""
    menu_bar = win32gui.GetMenu(hwnd)
    if not menu_bar:
        print("No menu bar found on the window.")
        return

    menu_count = win32gui.GetMenuItemCount(menu_bar)
    print(f"\n--- Enumerating Menus ({menu_count} top-level) ---")

    for i in range(menu_count):
        menu_text = get_menu_string(menu_bar, i).replace('&', '')
        submenu = win32gui.GetSubMenu(menu_bar, i)
        print(f"\nMenu {i}: '{menu_text}'")

        if submenu:
            item_count = win32gui.GetMenuItemCount(submenu)
            print(f"  Contains {item_count} items:")
            for j in range(item_count):
                item_text = get_menu_string(submenu, j)
                menu_id = win32gui.GetMenuItemID(submenu, j)
                
                id_str = f"(ID: {menu_id})" if menu_id != -1 and menu_id != 0 else ""
                
                sub_submenu = win32gui.GetSubMenu(submenu, j)
                if sub_submenu:
                    print(f"    Item {j}: '{item_text}' -> [Submenu]")
                    sub_item_count = win32gui.GetMenuItemCount(sub_submenu)
                    for k in range(sub_item_count):
                        sub_item_text = get_menu_string(sub_submenu, k)
                        sub_menu_id = win32gui.GetMenuItemID(sub_submenu, k)
                        sub_id_str = f"(ID: {sub_menu_id})" if sub_menu_id != -1 and sub_menu_id != 0 else ""
                        print(f"      - '{sub_item_text}' {sub_id_str}")
                else:
                    print(f"    Item {j}: '{item_text}' {id_str}")
        else:
            print("  (This top-level item is not a menu)")

def enumerate_child_controls(hwnd):
    """Enumerates all child controls (widgets) of a window."""
    child_windows = []
    def callback(child_hwnd, _):
        child_windows.append(child_hwnd)
        return True

    win32gui.EnumChildWindows(hwnd, callback, None)
    
    print(f"\n--- Enumerating Child Controls ({len(child_windows)} found) ---")
    if not child_windows:
        print("No child controls found.")
        return
        
    for i, child_hwnd in enumerate(child_windows):
        class_name = win32gui.GetClassName(child_hwnd)
        window_text = win32gui.GetWindowText(child_hwnd)
        control_id = win32gui.GetDlgCtrlID(child_hwnd)
        
        style = win32gui.GetWindowLong(child_hwnd, win32con.GWL_STYLE)
        is_visible = (style & win32con.WS_VISIBLE) != 0
        
        rect = win32gui.GetWindowRect(child_hwnd)
        
        print(f"\nControl {i}:")
        print(f"  - HWND:        {child_hwnd}")
        print(f"  - Class Name:  '{class_name}'")
        print(f"  - Text/Caption: '{window_text}'")
        print(f"  - Control ID:  {control_id}")
        print(f"  - Visible:     {is_visible}")
        print(f"  - Position:    (L: {rect[0]}, T: {rect[1]}, R: {rect[2]}, B: {rect[3]})")

# Main execution for pywin32 inspection
if 'hecras_pid' not in globals() or hecras_pid is None:
    print("ERROR: HEC-RAS process ID not found. Please run the previous cell to launch HEC-RAS first.")
else:
    print(f"Looking for HEC-RAS windows for process ID: {hecras_pid}")
    time.sleep(2)
    
    windows = get_windows_by_pid(hecras_pid)
    if not windows:
        print(f"Could not find any windows for process ID {hecras_pid}")
    else:
        hec_ras_hwnd, title = find_main_hecras_window(windows)
        if not hec_ras_hwnd:
            print("Could not identify the main HEC-RAS window from the found windows:")
            for hwnd, title in windows:
                print(f"  - {title} (HWND: {hwnd})")
        else:
            print(f"\nFound main HEC-RAS window: '{title}' (HWND: {hec_ras_hwnd})")
            print("="*60)
            
            enumerate_all_menus(hec_ras_hwnd)
            enumerate_child_controls(hec_ras_hwnd)
            
            print("\n" + "="*60)
            print("Inspection complete. The lists above show all menus and controls discoverable with pywin32.")



Looking for HEC-RAS windows for process ID: 398092
Could not identify the main HEC-RAS window from the found windows:
  - Loading File (HWND: 21174144)
  - RAS (HWND: 6292650)


In [7]:
# ==============================================================================
# Part 1b: Deeper Menu and Window Object Enumeration using ctypes
# ==============================================================================

import win32gui
import win32con
import win32api
import win32process
import ctypes
from ctypes import wintypes

# Define MENUITEMINFO structure using ctypes
class MENUITEMINFO(ctypes.Structure):
    _fields_ = [
        ("cbSize", wintypes.UINT),
        ("fMask", wintypes.UINT),
        ("fType", wintypes.UINT),
        ("fState", wintypes.UINT),
        ("wID", wintypes.UINT),
        ("hSubMenu", wintypes.HMENU),
        ("hbmpChecked", wintypes.HBITMAP),
        ("hbmpUnchecked", wintypes.HBITMAP),
        ("dwItemData", ctypes.POINTER(ctypes.c_ulong)),
        ("dwTypeData", wintypes.LPWSTR),
        ("cch", wintypes.UINT),
        ("hbmpItem", wintypes.HBITMAP)
    ]

def get_menu_string(menu_handle, pos):
    """Get menu item string at position"""
    buf_size = 256
    buf = ctypes.create_unicode_buffer(buf_size)
    user32 = ctypes.windll.user32
    result = user32.GetMenuStringW(menu_handle, pos, buf, buf_size, MF_BYPOSITION)
    if result:
        return buf.value
    return ""

def enumerate_menu_item_details(menu_handle, item_index):
    """Get detailed information about a menu item using ctypes"""
    # Create and initialize MENUITEMINFO structure
    mii = MENUITEMINFO()
    mii.cbSize = ctypes.sizeof(MENUITEMINFO)
    mii.fMask = win32con.MIIM_STATE | win32con.MIIM_ID | win32con.MIIM_TYPE | win32con.MIIM_SUBMENU
    
    # Call GetMenuItemInfo using ctypes
    user32 = ctypes.windll.user32
    result = user32.GetMenuItemInfoW(
        menu_handle, 
        item_index, 
        True,  # fByPosition
        ctypes.byref(mii)
    )
    
    if result:
        # Parse state flags
        state_flags = []
        if mii.fState & win32con.MFS_CHECKED:
            state_flags.append("CHECKED")
        if mii.fState & win32con.MFS_DISABLED:
            state_flags.append("DISABLED")
        if mii.fState & win32con.MFS_GRAYED:
            state_flags.append("GRAYED")
        if mii.fState & win32con.MFS_HILITE:
            state_flags.append("HIGHLIGHTED")
        if mii.fState & win32con.MFS_DEFAULT:
            state_flags.append("DEFAULT")
            
        # Parse type flags
        type_flags = []
        if mii.fType & win32con.MFT_STRING:
            type_flags.append("STRING")
        if mii.fType & win32con.MFT_SEPARATOR:
            type_flags.append("SEPARATOR")
        if mii.fType & win32con.MFT_BITMAP:
            type_flags.append("BITMAP")
        if mii.fType & win32con.MFT_OWNERDRAW:
            type_flags.append("OWNERDRAW")
        
        return {
            "id": mii.wID,
            "type_flags": type_flags,
            "state": state_flags,
            "text": get_menu_string(menu_handle, item_index),
            "has_submenu": bool(mii.hSubMenu)
        }
    else:
        # Fallback to simpler approach if GetMenuItemInfo fails
        try:
            menu_id = win32gui.GetMenuItemID(menu_handle, item_index)
            menu_state = win32gui.GetMenuState(menu_handle, item_index, win32con.MF_BYPOSITION)
            
            state_flags = []
            if menu_state & win32con.MF_CHECKED:
                state_flags.append("CHECKED")
            if menu_state & win32con.MF_DISABLED:
                state_flags.append("DISABLED")
            if menu_state & win32con.MF_GRAYED:
                state_flags.append("GRAYED")
            if menu_state & win32con.MF_SEPARATOR:
                state_flags.append("SEPARATOR")
            
            return {
                "id": menu_id if menu_id != -1 else None,
                "state": state_flags,
                "text": get_menu_string(menu_handle, item_index),
                "has_submenu": win32gui.GetSubMenu(menu_handle, item_index) is not None,
                "fallback": True
            }
        except Exception as e:
            return {
                "error": str(e),
                "text": get_menu_string(menu_handle, item_index)
            }

def enumerate_window_details(hwnd, indent=0):
    """Recursively enumerate window details including styles and extended styles"""
    if not hwnd:
        return
        
    # Get basic window info
    class_name = win32gui.GetClassName(hwnd)
    window_text = win32gui.GetWindowText(hwnd)
    
    # Get window styles
    style = win32gui.GetWindowLong(hwnd, win32con.GWL_STYLE)
    ex_style = win32gui.GetWindowLong(hwnd, win32con.GWL_EXSTYLE)
    
    # Get window metrics
    rect = win32gui.GetWindowRect(hwnd)
    
    # Print window details
    indent_str = "  " * indent
    print(f"{indent_str}Window Handle: {hwnd}")
    print(f"{indent_str}Class: {class_name}")
    print(f"{indent_str}Text: {window_text}")
    print(f"{indent_str}Position: {rect}")
    print(f"{indent_str}Style: 0x{style:08X}")
    print(f"{indent_str}Extended Style: 0x{ex_style:08X}")
    
    # Enumerate child windows recursively (limit depth to avoid too much output)
    if indent < 3:  # Limit recursion depth
        try:
            child_windows = []
            win32gui.EnumChildWindows(
                hwnd,
                lambda child_hwnd, windows: windows.append(child_hwnd) or True,
                child_windows
            )
            
            if child_windows:
                print(f"{indent_str}Children: {len(child_windows)} found")
                for child_hwnd in child_windows[:5]:  # Show first 5 children
                    print(f"{indent_str}---")
                    enumerate_window_details(child_hwnd, indent + 1)
                if len(child_windows) > 5:
                    print(f"{indent_str}... and {len(child_windows) - 5} more children")
        except Exception as e:
            print(f"{indent_str}Error enumerating children: {e}")

def enumerate_full_menu_tree(hwnd):
    """Enumerate complete menu tree with all available details"""
    menu_bar = win32gui.GetMenu(hwnd)
    if not menu_bar:
        print("No menu bar found")
        return
        
    print("\n=== Complete Menu Tree Analysis ===\n")
    
    menu_count = win32gui.GetMenuItemCount(menu_bar)
    for i in range(menu_count):
        menu_text = get_menu_string(menu_bar, i)
        menu_details = enumerate_menu_item_details(menu_bar, i)
        print(f"\nTop Level Menu {i}: {menu_text}")
        print(f"Details: {menu_details}")
        
        submenu = win32gui.GetSubMenu(menu_bar, i)
        if submenu:
            submenu_count = win32gui.GetMenuItemCount(submenu)
            print(f"Contains {submenu_count} items:")
            
            for j in range(min(submenu_count, 10)):  # Show first 10 items
                submenu_text = get_menu_string(submenu, j)
                submenu_details = enumerate_menu_item_details(submenu, j)
                print(f"  └─ Item {j}: {submenu_text}")
                print(f"     Details: {submenu_details}")
                
                # Check for sub-submenus
                sub_submenu = win32gui.GetSubMenu(submenu, j)
                if sub_submenu:
                    sub_count = win32gui.GetMenuItemCount(sub_submenu)
                    print(f"     Has submenu with {sub_count} items:")
                    for k in range(min(sub_count, 5)):  # Show first 5 sub-items
                        sub_text = get_menu_string(sub_submenu, k)
                        sub_details = enumerate_menu_item_details(sub_submenu, k)
                        print(f"       └─ Sub-item {k}: {sub_text}")
                        print(f"          Details: {sub_details}")
            
            if submenu_count > 10:
                print(f"  ... and {submenu_count - 10} more items")

# Main execution
if 'hecras_pid' in globals() and hecras_pid is not None:
    # Wait for windows to be ready
    import time
    time.sleep(1)
    
    # Find HEC-RAS windows
    windows = get_windows_by_pid(hecras_pid)
    hec_ras_hwnd, title = find_main_hecras_window(windows)
    
    if hec_ras_hwnd:
        print("\n" + "="*80)
        print("Performing detailed menu enumeration...")
        enumerate_full_menu_tree(hec_ras_hwnd)
        
        print("\n" + "="*80)
        print("Performing detailed window hierarchy enumeration...")
        print(f"Main window: {title}")
        enumerate_window_details(hec_ras_hwnd)
        
        print("\n" + "="*80)
        print("Detailed enumeration complete.")
    else:
        print("Could not find main HEC-RAS window")
else:
    print("HEC-RAS process ID not found. Please run the cell that launches HEC-RAS first.")


Performing detailed menu enumeration...

=== Complete Menu Tree Analysis ===


Top Level Menu 0: &File
Details: {'id': 1399855965, 'type_flags': [], 'state': [], 'text': '&File', 'has_submenu': True}
Contains 27 items:
  └─ Item 0: &New Project ...
     Details: {'id': 2, 'type_flags': [], 'state': [], 'text': '&New Project ...', 'has_submenu': False}
  └─ Item 1: &Open Project ...
     Details: {'id': 3, 'type_flags': [], 'state': [], 'text': '&Open Project ...', 'has_submenu': False}
  └─ Item 2: &Save Project
     Details: {'id': 4, 'type_flags': [], 'state': [], 'text': '&Save Project', 'has_submenu': False}
  └─ Item 3: Save Project &As ...
     Details: {'id': 5, 'type_flags': [], 'state': [], 'text': 'Save Project &As ...', 'has_submenu': False}
  └─ Item 4: &Rename Project Title ...
     Details: {'id': 6, 'type_flags': [], 'state': [], 'text': '&Rename Project Title ...', 'has_submenu': False}
  └─ Item 5: &Delete Project ...
     Details: {'id': 7, 'type_flags': [], 'state':

In [8]:
def enumerate_menu_item_details(menu_handle, item_index):
    """Get detailed information about a menu item"""
    # Since win32gui doesn't have MENUITEMINFO, we'll use a simpler approach
    try:
        # Get menu item ID
        menu_id = win32gui.GetMenuItemID(menu_handle, item_index)
        
        # Get menu state
        menu_state = win32gui.GetMenuState(menu_handle, item_index, win32con.MF_BYPOSITION)
        
        # Parse state flags
        state_flags = []
        if menu_state & win32con.MF_CHECKED:
            state_flags.append("CHECKED")
        if menu_state & win32con.MF_DISABLED:
            state_flags.append("DISABLED")
        if menu_state & win32con.MF_GRAYED:
            state_flags.append("GRAYED")
        if menu_state & win32con.MF_SEPARATOR:
            state_flags.append("SEPARATOR")
        
        # Get menu text
        text = get_menu_string(menu_handle, item_index)
        
        return {
            "id": menu_id if menu_id != -1 else None,
            "state": state_flags,
            "text": text,
            "has_submenu": win32gui.GetSubMenu(menu_handle, item_index) is not None
        }
    except Exception as e:
        return {
            "error": str(e),
            "text": get_menu_string(menu_handle, item_index)
        }

In [9]:
# ==============================================================================
# Part 2: Interacting with 64-bit Processes using pywinauto
# ==============================================================================
print("\n--- pywinauto Example ---")
print("This section shows how to inspect an application like RAS Mapper using pywinauto.")
print("You may need to install it first: !pip install pywinauto")
try:
    import pywinauto
    from pywinauto import timings, findwindows
    print("pywinauto is installed.")
    
    print("\nAttempting to find a RAS Mapper window to demonstrate pywinauto...")
    
    try:
        # Connect to RAS Mapper window (title might vary slightly by version)
        # Using win32 backend as RAS Mapper is a mix of technologies
        app = pywinauto.Application(backend="win32").connect(title_re=".*RAS Mapper.*", timeout=5)
        
        ras_mapper_window = app.window(title_re=".*RAS Mapper.*")
        
        print("\nSuccessfully connected to RAS Mapper!")
        print("Now, listing all its controls using pywinauto's print_control_identifiers():")
        
        # This will print a tree of all interactable controls.
        # It's a very useful function for exploration.
        ras_mapper_window.print_control_identifiers()
        
    except (findwindows.ElementNotFoundError, timings.TimeoutError):
        print("\nCould not find a running RAS Mapper window.")
        print("To run this for real, open HEC-RAS and then RAS Mapper, then execute this cell again.")
        print("Example code to list controls once connected:")
        print("  from pywinauto import Application")
        print("  app = Application(backend='uia').connect(title_re='.*RAS Mapper.*') # uia backend might also work")
        print("  ras_mapper_window = app.window(title_re='.*RAS Mapper.*')")
        print("  ras_mapper_window.print_control_identifiers()")

except ImportError:
    print("\npywinauto is not installed. Please install it to run this example:")
    print("In a new cell, run: !pip install pywinauto")


--- pywinauto Example ---
This section shows how to inspect an application like RAS Mapper using pywinauto.
You may need to install it first: !pip install pywinauto
pywinauto is installed.

Attempting to find a RAS Mapper window to demonstrate pywinauto...

Could not find a running RAS Mapper window.
To run this for real, open HEC-RAS and then RAS Mapper, then execute this cell again.
Example code to list controls once connected:
  from pywinauto import Application
  app = Application(backend='uia').connect(title_re='.*RAS Mapper.*') # uia backend might also work
  ras_mapper_window = app.window(title_re='.*RAS Mapper.*')
  ras_mapper_window.print_control_identifiers()


In [10]:
# ==============================================================================
# Part 1b: Deeper Menu and Window Object Enumeration using ctypes
# ==============================================================================

import win32gui
import win32con
import win32api
import win32process
import ctypes
from ctypes import wintypes

# Define MENUITEMINFO structure using ctypes
class MENUITEMINFO(ctypes.Structure):
    _fields_ = [
        ("cbSize", wintypes.UINT),
        ("fMask", wintypes.UINT),
        ("fType", wintypes.UINT),
        ("fState", wintypes.UINT),
        ("wID", wintypes.UINT),
        ("hSubMenu", wintypes.HMENU),
        ("hbmpChecked", wintypes.HBITMAP),
        ("hbmpUnchecked", wintypes.HBITMAP),
        ("dwItemData", ctypes.POINTER(ctypes.c_ulong)),
        ("dwTypeData", wintypes.LPWSTR),
        ("cch", wintypes.UINT),
        ("hbmpItem", wintypes.HBITMAP)
    ]

def get_menu_string(menu_handle, pos):
    """Get menu item string at position"""
    buf_size = 256
    buf = ctypes.create_unicode_buffer(buf_size)
    user32 = ctypes.windll.user32
    result = user32.GetMenuStringW(menu_handle, pos, buf, buf_size, MF_BYPOSITION)
    if result:
        return buf.value
    return ""

def enumerate_menu_item_details(menu_handle, item_index):
    """Get detailed information about a menu item using ctypes"""
    # Create and initialize MENUITEMINFO structure
    mii = MENUITEMINFO()
    mii.cbSize = ctypes.sizeof(MENUITEMINFO)
    mii.fMask = win32con.MIIM_STATE | win32con.MIIM_ID | win32con.MIIM_TYPE | win32con.MIIM_SUBMENU
    
    # Call GetMenuItemInfo using ctypes
    user32 = ctypes.windll.user32
    result = user32.GetMenuItemInfoW(
        menu_handle, 
        item_index, 
        True,  # fByPosition
        ctypes.byref(mii)
    )
    
    if result:
        # Parse state flags
        state_flags = []
        if mii.fState & win32con.MFS_CHECKED:
            state_flags.append("CHECKED")
        if mii.fState & win32con.MFS_DISABLED:
            state_flags.append("DISABLED")
        if mii.fState & win32con.MFS_GRAYED:
            state_flags.append("GRAYED")
        if mii.fState & win32con.MFS_HILITE:
            state_flags.append("HIGHLIGHTED")
        if mii.fState & win32con.MFS_DEFAULT:
            state_flags.append("DEFAULT")
            
        # Parse type flags
        type_flags = []
        if mii.fType & win32con.MFT_STRING:
            type_flags.append("STRING")
        if mii.fType & win32con.MFT_SEPARATOR:
            type_flags.append("SEPARATOR")
        if mii.fType & win32con.MFT_BITMAP:
            type_flags.append("BITMAP")
        if mii.fType & win32con.MFT_OWNERDRAW:
            type_flags.append("OWNERDRAW")
        
        return {
            "id": mii.wID,
            "type_flags": type_flags,
            "state": state_flags,
            "text": get_menu_string(menu_handle, item_index),
            "has_submenu": bool(mii.hSubMenu)
        }
    else:
        # Fallback to simpler approach if GetMenuItemInfo fails
        try:
            menu_id = win32gui.GetMenuItemID(menu_handle, item_index)
            menu_state = win32gui.GetMenuState(menu_handle, item_index, win32con.MF_BYPOSITION)
            
            state_flags = []
            if menu_state & win32con.MF_CHECKED:
                state_flags.append("CHECKED")
            if menu_state & win32con.MF_DISABLED:
                state_flags.append("DISABLED")
            if menu_state & win32con.MF_GRAYED:
                state_flags.append("GRAYED")
            if menu_state & win32con.MF_SEPARATOR:
                state_flags.append("SEPARATOR")
            
            return {
                "id": menu_id if menu_id != -1 else None,
                "state": state_flags,
                "text": get_menu_string(menu_handle, item_index),
                "has_submenu": win32gui.GetSubMenu(menu_handle, item_index) is not None,
                "fallback": True
            }
        except Exception as e:
            return {
                "error": str(e),
                "text": get_menu_string(menu_handle, item_index)
            }

def enumerate_window_details(hwnd, indent=0):
    """Recursively enumerate window details including styles and extended styles"""
    if not hwnd:
        return
        
    # Get basic window info
    class_name = win32gui.GetClassName(hwnd)
    window_text = win32gui.GetWindowText(hwnd)
    
    # Get window styles
    style = win32gui.GetWindowLong(hwnd, win32con.GWL_STYLE)
    ex_style = win32gui.GetWindowLong(hwnd, win32con.GWL_EXSTYLE)
    
    # Get window metrics
    rect = win32gui.GetWindowRect(hwnd)
    
    # Print window details
    indent_str = "  " * indent
    print(f"{indent_str}Window Handle: {hwnd}")
    print(f"{indent_str}Class: {class_name}")
    print(f"{indent_str}Text: {window_text}")
    print(f"{indent_str}Position: {rect}")
    print(f"{indent_str}Style: 0x{style:08X}")
    print(f"{indent_str}Extended Style: 0x{ex_style:08X}")
    
    # Enumerate child windows recursively (limit depth to avoid too much output)
    if indent < 3:  # Limit recursion depth
        try:
            child_windows = []
            win32gui.EnumChildWindows(
                hwnd,
                lambda child_hwnd, windows: windows.append(child_hwnd) or True,
                child_windows
            )
            
            if child_windows:
                print(f"{indent_str}Children: {len(child_windows)} found")
                for child_hwnd in child_windows[:5]:  # Show first 5 children
                    print(f"{indent_str}---")
                    enumerate_window_details(child_hwnd, indent + 1)
                if len(child_windows) > 5:
                    print(f"{indent_str}... and {len(child_windows) - 5} more children")
        except Exception as e:
            print(f"{indent_str}Error enumerating children: {e}")

def enumerate_full_menu_tree(hwnd):
    """Enumerate complete menu tree with all available details"""
    menu_bar = win32gui.GetMenu(hwnd)
    if not menu_bar:
        print("No menu bar found")
        return
        
    print("\n=== Complete Menu Tree Analysis ===\n")
    
    menu_count = win32gui.GetMenuItemCount(menu_bar)
    for i in range(menu_count):
        menu_text = get_menu_string(menu_bar, i)
        menu_details = enumerate_menu_item_details(menu_bar, i)
        print(f"\nTop Level Menu {i}: {menu_text}")
        print(f"Details: {menu_details}")
        
        submenu = win32gui.GetSubMenu(menu_bar, i)
        if submenu:
            submenu_count = win32gui.GetMenuItemCount(submenu)
            print(f"Contains {submenu_count} items:")
            
            for j in range(min(submenu_count, 10)):  # Show first 10 items
                submenu_text = get_menu_string(submenu, j)
                submenu_details = enumerate_menu_item_details(submenu, j)
                print(f"  └─ Item {j}: {submenu_text}")
                print(f"     Details: {submenu_details}")
                
                # Check for sub-submenus
                sub_submenu = win32gui.GetSubMenu(submenu, j)
                if sub_submenu:
                    sub_count = win32gui.GetMenuItemCount(sub_submenu)
                    print(f"     Has submenu with {sub_count} items:")
                    for k in range(min(sub_count, 5)):  # Show first 5 sub-items
                        sub_text = get_menu_string(sub_submenu, k)
                        sub_details = enumerate_menu_item_details(sub_submenu, k)
                        print(f"       └─ Sub-item {k}: {sub_text}")
                        print(f"          Details: {sub_details}")
            
            if submenu_count > 10:
                print(f"  ... and {submenu_count - 10} more items")

# Main execution
if 'hecras_pid' in globals() and hecras_pid is not None:
    # Wait for windows to be ready
    import time
    time.sleep(1)
    
    # Find HEC-RAS windows
    windows = get_windows_by_pid(hecras_pid)
    hec_ras_hwnd, title = find_main_hecras_window(windows)
    
    if hec_ras_hwnd:
        print("\n" + "="*80)
        print("Performing detailed menu enumeration...")
        enumerate_full_menu_tree(hec_ras_hwnd)
        
        print("\n" + "="*80)
        print("Performing detailed window hierarchy enumeration...")
        print(f"Main window: {title}")
        enumerate_window_details(hec_ras_hwnd)
        
        print("\n" + "="*80)
        print("Detailed enumeration complete.")
    else:
        print("Could not find main HEC-RAS window")
else:
    print("HEC-RAS process ID not found. Please run the cell that launches HEC-RAS first.")


Performing detailed menu enumeration...

=== Complete Menu Tree Analysis ===


Top Level Menu 0: &File
Details: {'id': 1399855965, 'type_flags': [], 'state': [], 'text': '&File', 'has_submenu': True}
Contains 27 items:
  └─ Item 0: &New Project ...
     Details: {'id': 2, 'type_flags': [], 'state': [], 'text': '&New Project ...', 'has_submenu': False}
  └─ Item 1: &Open Project ...
     Details: {'id': 3, 'type_flags': [], 'state': [], 'text': '&Open Project ...', 'has_submenu': False}
  └─ Item 2: &Save Project
     Details: {'id': 4, 'type_flags': [], 'state': [], 'text': '&Save Project', 'has_submenu': False}
  └─ Item 3: Save Project &As ...
     Details: {'id': 5, 'type_flags': [], 'state': [], 'text': 'Save Project &As ...', 'has_submenu': False}
  └─ Item 4: &Rename Project Title ...
     Details: {'id': 6, 'type_flags': [], 'state': [], 'text': '&Rename Project Title ...', 'has_submenu': False}
  └─ Item 5: &Delete Project ...
     Details: {'id': 7, 'type_flags': [], 'state':

# This provides the basic building blocks for GUI-driven automations for HEC-RAS without the HECRASController